# Sequence Models
This tutorial is closely based off of [Sequence Models and Long-Short Term Memory Networks](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html) tutorial on the Pytorch webpage. The original author is Robert Guthrie.

In this tutorial, you will learn about LSTM neural networks and see an example of how they can be used to recognize parts of speech.

In [1]:
import math
import os
import jdc
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import Image

torch.manual_seed(1)

## Introduction
### What is an LSTM?
LSTM stands for Long Short-Term Memory. It is a type of recurrent neural network (RNN). RNNs are neural networks that can learn relationships within sequences, such as time series or sentences. A normal RNN has difficulty learning relationships when there is a large separation within the sequence. For example, consider the sequence: _The humidity is very high. Today it is going to_ _**rain**_. A regular RNN might not do a good job predicting the word **rain**. But, an LSTM could do a much better job with this prediction.

<img src="https://raw.githubusercontent.com/PythonWorkshop/intro-to-nlp-with-pytorch/master/images/lstm_flow.png" width=50%>

The network can learn sequences of information and make predictions based off of what it learns. It is a type of recurrent neural network. The LSTM cell has a state, which gets updated as the network trains. It is this state that allows the network to remember.

Here is a basic diagram of an LSTM:

<img src="https://raw.githubusercontent.com/PythonWorkshop/intro-to-nlp-with-pytorch/master/images/lstm_inner_workings.png" width=50%>

### LSTM Components
An LSTM replaces the recurrent model of an RNN with a module that uses a cell state. Gates, composed of sigmoid activation functions, decide what information to keep within the cell state and what information to remove. These gates go by the following names:
* Forget Gate
* Input Gate
* Output Gate

#### Forget Gate
Here the network decides what information to remove. Continuing with our previous example, let's say the network receives as input low humidity, the network might want to then adjust the rain likelihood for today to low. The forget gate then removes the current state of rain likelihood.

#### Input Gate.
The input gate decides what information to add to the cell state. In our example, it would replace the previous rain likelihood with the new likelihood for dry weather.

#### Output Gate
This gate returns a filtered version of the cell state. For example, it might keep track of the humidities magnitude to help the network decide whether to predict a large storm or just light rain.

### Very Simple Example 1
Before approaching the main example for this tutorial, let's see a very brief example of how to create an LSTM cell in Pytorch and pass information through it. One way to do this is using a **`for`** loop:

In [2]:
my_tensor = torch.randn(1, 1, 3)

In [3]:
my_tensor

tensor([[[0.6614, 0.2669, 0.0617]]])

In [4]:
my_tensor.view(1, 1, -1)

tensor([[[0.6614, 0.2669, 0.0617]]])

In [5]:
lstm = nn.LSTM(3, 3)  # Input dimension is 3, output dimension is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # Make a sequence of length 5

# Initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # After each time step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    print(out)
    print(hidden)
    print('')

tensor([[[-0.1960,  0.4022,  0.0955]]], grad_fn=<StackBackward>)
(tensor([[[-0.1960,  0.4022,  0.0955]]], grad_fn=<StackBackward>), tensor([[[-0.6544,  0.5604,  0.7848]]], grad_fn=<StackBackward>))

tensor([[[-0.2759,  0.3144,  0.1696]]], grad_fn=<StackBackward>)
(tensor([[[-0.2759,  0.3144,  0.1696]]], grad_fn=<StackBackward>), tensor([[[-0.6518,  0.4700,  0.5978]]], grad_fn=<StackBackward>))

tensor([[[-0.3945,  0.1052,  0.1429]]], grad_fn=<StackBackward>)
(tensor([[[-0.3945,  0.1052,  0.1429]]], grad_fn=<StackBackward>), tensor([[[-0.6290,  0.2144,  0.2851]]], grad_fn=<StackBackward>))

tensor([[[-0.3154,  0.0146,  0.1033]]], grad_fn=<StackBackward>)
(tensor([[[-0.3154,  0.0146,  0.1033]]], grad_fn=<StackBackward>), tensor([[[-0.6353,  0.0234,  0.2475]]], grad_fn=<StackBackward>))

tensor([[[-0.2813,  0.2131,  0.1043]]], grad_fn=<StackBackward>)
(tensor([[[-0.2813,  0.2131,  0.1043]]], grad_fn=<StackBackward>), tensor([[[-0.7320,  0.2745,  0.2935]]], grad_fn=<StackBackward>))



In [6]:
print('out: {}'.format(out))
print('hidden: {}'.format(hidden))

out: tensor([[[-0.2813,  0.2131,  0.1043]]], grad_fn=<StackBackward>)
hidden: (tensor([[[-0.2813,  0.2131,  0.1043]]], grad_fn=<StackBackward>), tensor([[[-0.7320,  0.2745,  0.2935]]], grad_fn=<StackBackward>))


### Very Simple Example 2
Instead of creating an LSTM using a **`for`** loop, we can use Pytorch's **`cat`** function to string together each layer of the LSTM.

In [7]:
lstm = nn.LSTM(3, 3)  # Input dimension is 3, output dimension is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # Make a sequence of length 5

inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # Clean out hidden state
out, hidden = lstm(inputs, hidden)  # out = all states, hidden = last state and last cell state

In [8]:
print('out: {}'.format(out))
print('hidden: {}'.format(hidden))

out: tensor([[[-0.3722,  0.0537,  0.2904]],

        [[-0.4462, -0.0048,  0.1626]],

        [[-0.3391, -0.0817, -0.0695]],

        [[-0.3841,  0.0629, -0.1056]],

        [[-0.4056,  0.0588, -0.1469]]], grad_fn=<StackBackward>)
hidden: (tensor([[[-0.4056,  0.0588, -0.1469]]], grad_fn=<StackBackward>), tensor([[[-1.0460,  0.0934, -0.3533]]], grad_fn=<StackBackward>))


## Example: An LSTM for Part-of-Speech Tagging
In this example, a model will be created that can predict the part-of-speech for each word in a sentence. TODO: Show rolled out LSTM, labeling each word and part of speech.

### Prepare the data:
Let's begin by creating a dataset for training. The dataset will consist of a list of sequences. Each sequence will contain two lists. The first list is a sentence split up into words. The second list contains grammmer identifiers for each word in the sentence.

In [9]:
training_data = [
    ("The dog ate the apple.".split(), ["Determiner", "Noun", "Verb", "Determiner", "Noun"]),
    ("Everybody read that book.".split(), ["Noun", "Verb", "Determiner", "Noun"])
]
training_sentences = [training_data[x][0] for x in range(len(training_data))]

Since Pytorch only understands numbers, we need to map strings, such as the words in the training set, to integers. The following lines of code create a dictionary with this mapping:

In [10]:
word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print('word_to_ix: {}'.format(word_to_ix))

word_to_ix: {'the': 3, 'that': 7, 'apple.': 4, 'read': 6, 'dog': 1, 'ate': 2, 'book.': 8, 'The': 0, 'Everybody': 5}


We also need to map the parts-of-speech tags to integers:

In [11]:
# Tags to integers
tag_to_ix = {"Determiner": 0, "Noun": 1, "Verb": 2}

A third dictionary is used to map the integers back to parts-of-speech.

In [12]:
# Integers to tags
ix_to_tag = {0: "Determiner", 1: "Noun", 2: "Verb"}

### Set Hyperparameters

In [13]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
LEARNING_RATE = 0.1
NUM_EPOCHS = 300

### Create the model
We will define the model by creating a Python class object. This class will inherit the nn.Module class from Pytorch, which will allow us to easily use the neural network classes defined in Pytorch.

The **`LSTMTagger`** class will take in four values, the embedding dimension, the number of hidden dimensions, the vocabulary size, and the size of the tag set.

In [14]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # LSTM: Inputs are embeddings, outputs are hidden states
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # Linear layer maps hidden space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

We will now create a function to initialize the hidden states.

In [15]:
%%add_to LSTMTagger
def init_hidden(self):
    """
    Initialize the hidden state. The axes correspond to (num_layers, minibatch_size, hidden_dim).
    """
    return (torch.zeros(1, 1, self.hidden_dim),
            torch.zeros(1, 1, self.hidden_dim))

Now we define a function to make a forward pass through the recurrent LSTM network. It will return the predict tag values given an input sentence.

In [16]:
%%add_to LSTMTagger
def forward(self, sentence):
    """
    Make a forward pass through the LSTM.
    
    :param sentence: The input sentence.
    :type sentence: list
    :return: A Tensor of tag scores.
    :rtype: Tensor
    """
    embeds = self.word_embeddings(sentence)
    lstm_out, self.hidden = self.lstm(
        embeds.view(len(sentence), 1, -1), self.hidden)
    tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
    tag_scores = F.log_softmax(tag_space, dim=1)
    return tag_scores

### Helper Function
This helper function will be used to map either words or tags to integers, using the previously defined dictionaries (**`tag_to_ix`**, **`ix_to_tag`**).

In [17]:
def prepare_sequence(seq, to_ix):
    """
    Convert words or tags to intigers and return a Pytorch tensor.
    :param seq: Sequence of words.
    :type seq: list
    :param to_ix: Dictionary mapping words or tags to intigers.
    :return: A Pytorch tensor of indices.
    :rtype: Tensor
    """
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

### Train the model:

Create the LSTM Pytorch model using the hyperparameters defined above.

In [18]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))

Define the loss function. In this case, we will be using a negative log likelihood function, which is useful in classification problems.

In [19]:
loss_function = nn.NLLLoss()

We can illustrate negative log likelihood in the following diagram:

<img src="https://raw.githubusercontent.com/PythonWorkshop/intro-to-nlp-with-pytorch/master/images/nll_loss.png" width=50%>

We will train the model using stochastic gradient descent.

In [20]:
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

Let's run the model before any training has been done and store the scores to a **`list`**. We will then compare these scores with the scores after training.

In [21]:
store_initial_probabilities = []
store_initial_predictions = []
with torch.no_grad():
    for sentence in training_sentences:
        inputs = prepare_sequence(sentence, word_to_ix)
        tag_scores = model(inputs)
        tag_probabilities = tag_scores.exp()
        max_values, max_indices = torch.max(tag_probabilities, 1)
        initial_prediction = [ix_to_tag[x] for x in max_indices.numpy()]
        store_initial_predictions.append(initial_prediction)
        store_initial_probabilities.append(tag_probabilities)

Now, we will train the model.

In [22]:
for epoch in range(NUM_EPOCHS):
    for sentence, tags in training_data:
        # Set gradients equal to zero after each intance
        model.zero_grad()
        
        # Initialize hidden state of LSTM after each intance
        model.hidden = model.init_hidden()
        
        # Turn inputs into tensors of word indices
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        # Run forward pass
        tag_scores = model(sentence_in)
        
        # Compute the loss, gradients, and update the parameters
        loss = loss_function(tag_scores, targets)
        
        # Perform backward pass
        loss.backward()
        
        # Update model parameters
        optimizer.step()

Our model has now finished training. Let's print out some statistics to show how well the model training performed. TODO: Understand what numbers for scores mean and create diagram.

In [23]:
# Print out the scores after training the model
store_initial_probabilities.reverse()
store_initial_predictions.reverse()
with torch.no_grad():
    for sentence in training_sentences:
        inputs = prepare_sequence(sentence, word_to_ix)
        tag_scores = model(inputs)
        tag_probabilities = tag_scores.exp()
        max_values, max_indices = torch.max(tag_probabilities, 1)
        predictions = [ix_to_tag[x] for x in max_indices.numpy()]
        
        print('Before training:')
        print(' - initial probabilities: {}'.format(store_initial_probabilities.pop()))
        print(' - sentence: {}'.format(' '.join(sentence)))
        print(' - predicition: {}'.format(store_initial_predictions.pop()))
        print('After training:')
        print(' - final probabilities: {}'.format(tag_probabilities))
        print(' - sentence: {}'.format(' '.join(sentence)))
        print(' - prediction: {}'.format(predictions))
        print('')

Before training:
 - initial probabilities: tensor([[0.4544, 0.2701, 0.2754],
        [0.4564, 0.2645, 0.2790],
        [0.4570, 0.2712, 0.2719],
        [0.4797, 0.2384, 0.2819],
        [0.4843, 0.2447, 0.2709]])
 - sentence: The dog ate the apple.
 - predicition: ['Determiner', 'Determiner', 'Determiner', 'Determiner', 'Determiner']
After training:
 - final probabilities: tensor([[0.7180, 0.1744, 0.1075],
        [0.0044, 0.9745, 0.0212],
        [0.0186, 0.0261, 0.9553],
        [0.9473, 0.0487, 0.0040],
        [0.0528, 0.9386, 0.0086]])
 - sentence: The dog ate the apple.
 - prediction: ['Determiner', 'Noun', 'Verb', 'Determiner', 'Noun']

Before training:
 - initial probabilities: tensor([[0.4776, 0.2407, 0.2817],
        [0.4781, 0.2576, 0.2643],
        [0.4873, 0.2377, 0.2751],
        [0.4669, 0.2237, 0.3094]])
 - sentence: Everybody read that book.
 - predicition: ['Determiner', 'Determiner', 'Determiner', 'Determiner']
After training:
 - final probabilities: tensor([[0.0160

### What do the scores mean?
The scores are used to predict the parts-of-speech for each word in a sentence. A corresponding list of of possible parts-of-speech is assigned to each word. This list is the same for all data passed through the model. For example:

In [24]:
print('Let us take the sentence: {}'.format(' '.join(training_sentences[0])))
print('For the word "{}" the list of possible parts-of-speech are: {}'.format(training_sentences[0][0], [x for x in ix_to_tag.values()]))

Let us take the sentence: The dog ate the apple.
For the word "The" the list of possible parts-of-speech are: ['Determiner', 'Noun', 'Verb']


The model assigns a scores to each part-of-speech in the list. The prediction is then the part-of-speech with the highest score. If the model makes a correct prediction, then `Determiner` will have the highest score.